In [9]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [10]:
for filename in os.listdir("../data/RawData/BLS"):
    print(filename.split(".")[0])

banana
beef
bread
chicken
coffee
eggs
electricity
flour
iceCream
rice
unleadedGasoline


In [11]:
def toQuarter(month):
    if month>=1 and month<=3:
        return 1
    elif month > 3 and month <=6:
        return 2
    elif month > 6 and month <= 9:
        return 3
    else: 
        return 4

In [12]:
def formatData(filename):
    data = pd.read_csv('../data/RawData/BLS/'+filename,usecols=["Year","Period","Value"])
    data=data[data["Year"]>=1992]
    data.Period = data.Period.apply(lambda x: x[1:]).astype("int")
    data["Quarter"] = data.Period.apply(toQuarter)
    data = data.drop(columns=["Period"])
    data = data.groupby(["Year","Quarter"]).mean()
    return data

In [13]:
def combine(df,left_on_col = ["YEAR FORECAST MADE","QUARTER"]):
    returnFrame = df.copy()
    for filename in os.listdir("../data/RawData/BLS"):
        to_merge = formatData(filename).reset_index(level=[0,1])
        to_merge.rename(columns={"Value":filename.split(".")[0]},inplace=True)
        returnFrame = returnFrame.merge(to_merge,how="outer",left_on=left_on_col,right_on=["Year","Quarter"],suffixes=('', '_drop'))
    #returnFrame[['banana', 'beef', 'bread', 'chicken', 'coffee', 'eggs','electricity', 'flour', 'iceCream', 'rice', 'unleadedGasoline']] = returnFrame[['banana', 'beef', 'bread', 'chicken', 'coffee', 'eggs','electricity', 'flour', 'iceCream', 'rice', 'unleadedGasoline']].fillna(method="ffill")
    returnFrame = returnFrame[returnFrame.columns.drop(returnFrame.filter(regex='drop').columns)]
    return returnFrame.drop(columns=["Year","Quarter"])
    

In [14]:
training = pd.read_csv("../data/TrainingData/fulldata.csv")
training

,YEAR FORECAST MADE,QUARTER,YEAR BEING FORECAST,INDICATOR,FORECASTER ID,INDUSTRY,Year.ID.ForecastYear.Quarter,Indicator.Year.ID.ForecastYear.Quarter,BIN 1,BIN 2,...,bin_value_sum,TDIST,actual,ACTUAL_BIN,MAX,NUM_OF_MAX,ACTUAL_CONF,HIT,pred_average,pred_var
0,1992,1,1992,RealGDP,20,2,1992-20-1992-1,RealGDP-1992-20-1992-1,0.0,0.0,...,100.0,0.75,3.52244,4.0,0.5,2.0,0.0,0.0,1.000,0.250000
1,1992,1,1992,RealGDP,30,3,1992-30-1992-1,RealGDP-1992-30-1992-1,0.0,0.0,...,100.0,0.75,3.52244,4.0,0.6,1.0,0.6,1.0,3.300,0.360000
2,1992,1,1992,RealGDP,35,2,1992-35-1992-1,RealGDP-1992-35-1992-1,0.0,0.0,...,100.0,0.75,3.52244,4.0,0.5,1.0,0.1,0.0,2.100,1.240000
3,1992,1,1992,RealGDP,40,2,1992-40-1992-1,RealGDP-1992-40-1992-1,NaN,NaN,...,NaN,0.75,3.52244,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1992,1,1992,RealGDP,60,2,1992-60-1992-1,RealGDP-1992-60-1992-1,0.0,0.0,...,100.0,0.75,3.52244,4.0,0.6,1.0,0.0,0.0,1.150,0.427500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31041,2021,2,2024,Unemployment,586,2,2024-586-2021-2,Unemployment-2024-586-2021-2,NaN,NaN,...,NaN,3.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31042,2021,2,2024,Unemployment,587,2,2024-587-2021-2,Unemployment-2024-587-2021-2,0.0,0.0,...,100.0,3.50,NaN,NaN,0.5,1.0,NaN,NaN,5.125,0.203125
31043,2021,2,2024,Unemployment,588,1,2024-588-2021-2,Unemployment-2024-588-2021-2,NaN,NaN,...,NaN,3.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31044,2021,2,2024,Unemployment,589,2,2024-589-2021-2,Unemployment-2024-589-2021-2,0.0,0.0,...,100.0,3.50,NaN,NaN,0.9,1.0,NaN,NaN,5.175,0.050625


In [15]:
newTraining = combine(training)


In [16]:
newTraining

,YEAR FORECAST MADE,QUARTER,YEAR BEING FORECAST,INDICATOR,FORECASTER ID,INDUSTRY,Year.ID.ForecastYear.Quarter,Indicator.Year.ID.ForecastYear.Quarter,BIN 1,BIN 2,...,beef,bread,chicken,coffee,eggs,electricity,flour,iceCream,rice,unleadedGasoline
0,1992,1,1992,RealGDP,20,2,1992-20-1992-1,RealGDP-1992-20-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
1,1992,1,1992,RealGDP,30,3,1992-30-1992-1,RealGDP-1992-30-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
2,1992,1,1992,RealGDP,35,2,1992-35-1992-1,RealGDP-1992-35-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
3,1992,1,1992,RealGDP,40,2,1992-40-1992-1,RealGDP-1992-40-1992-1,NaN,NaN,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
4,1992,1,1992,RealGDP,60,2,1992-60-1992-1,RealGDP-1992-60-1992-1,0.0,0.0,...,1.577667,0.734333,0.862000,2.662333,0.888,0.085000,0.241333,2.605000,0.515333,1.061667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31041,2021,2,2024,Unemployment,586,2,2024-586-2021-2,Unemployment-2024-586-2021-2,NaN,NaN,...,4.184667,1.510333,1.491667,4.641000,1.629,0.140333,0.410667,4.849667,0.788000,2.988333
31042,2021,2,2024,Unemployment,587,2,2024-587-2021-2,Unemployment-2024-587-2021-2,0.0,0.0,...,4.184667,1.510333,1.491667,4.641000,1.629,0.140333,0.410667,4.849667,0.788000,2.988333
31043,2021,2,2024,Unemployment,588,1,2024-588-2021-2,Unemployment-2024-588-2021-2,NaN,NaN,...,4.184667,1.510333,1.491667,4.641000,1.629,0.140333,0.410667,4.849667,0.788000,2.988333
31044,2021,2,2024,Unemployment,589,2,2024-589-2021-2,Unemployment-2024-589-2021-2,0.0,0.0,...,4.184667,1.510333,1.491667,4.641000,1.629,0.140333,0.410667,4.849667,0.788000,2.988333


In [31]:
newTraining.to_csv("../data/TrainingData/trainingWithItems.csv",index=False)

In [ ]:
quarterFred= pd.read_csv("../data/CleanData/QuarterlyFred.csv",index_col=0)
quarterFred["DATE"] = quarterFred["DATE"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
quarterFred["Year"] = quarterFred["DATE"].apply(lambda x:x.year)
quarterFred["Month"] = quarterFred["DATE"].apply(lambda y:y.month)
quarterFred["Quarter"] = quarterFred["Month"].apply(toQuarter)
quarterFred.drop(columns=["Month"],inplace=True)
quarterFred.head()

In [ ]:
newQuarterly = combine(quarterFred,["Year","Quarter"])
newQuarterly.head()
#newQuarterly.to_csv("../data/CleanData/QuarterlyFredWithItems.csv",index=False)